In [4]:
#import libraries
import numpy as np
import cv2 
import os
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.utils.np_utils import to_categorical

In [5]:
#Read CSV File and shuffle rows
data=pd.read_csv("Labels.csv")
data = data.sample(frac=1).reset_index(drop=True)

In [6]:
total_classes = np.unique(data['Label']).shape[0]
lb = LabelEncoder()
lb.fit(data['Label'])
lbs = lb.transform(data['Label'])

In [10]:
images = np.zeros((data.shape[0],200,200,3))
labels = np.ones((data.shape[0],))

In [11]:
for i in range(data.shape[0]):
    img = cv2.imread('./faces/'+data.loc[i,'File'])
    images[i,:,:,:] = img
    print(str(data.loc[i,'File'])+' - '+str(lbs[i]))
    labels[i] = lbs[i]

batch3_55.png - 2
batch2selfies10.png - 1
batch3selfies8.png - 1
batch1_4.png - 0
batch2selfies4.png - 1
batch3_58.png - 0
36.png - 2
batch1selfies7.png - 1
batch4_72.png - 3
batch4_25.png - 1
batch4_48.png - 1
batch2_14.png - 1
selfies8.png - 1
batch4_61.png - 3
batch4_46.png - 2
batch3_53.png - 0
batch2_56.png - 1
batch1selfies11.png - 1
batch4_47.png - 2
selfies2.png - 1
batch4_62.png - 3
batch1_72.png - 3
batch3_59.png - 3
batch2_32.png - 2
batch2_49.png - 0
selfies11.png - 1
batch1_9.png - 0
batch4_63.png - 3
batch1_26.png - 1
1.png - 0
batch1_17.png - 1
batch1_64.png - 3
batch3selfies2.png - 1
batch1selfies4.png - 1
batch3selfies11.png - 1
batch3selfies13.png - 2
batch1_65.png - 3
batch2_4.png - 0
batch4_29.png - 1
batch2_42.png - 2
batch2_25.png - 1
batch2selfies8.png - 1
batch4_1.png - 0
selfies10.png - 1
batch1_49.png - 0
batch1_23.png - 1
59.png - 3
71.png - 3
batch3_9.png - 0
15.png - 1
batch2_45.png - 2
batch1selfies6.png - 1
batch1_14.png - 1
batch1selfies12.png - 2
batch2

In [13]:
labels.shape

(436,)

In [24]:
encoded_labels = to_categorical(labels, num_classes=total_classes)

In [25]:
def convLayer(X,num_input_channels,num_filters,filter_shape,pool_shape,name):

    conv_filt_shape = [filter_shape[0], filter_shape[1], num_input_channels,num_filters]
    b1 = tf.Variable(tf.zeros([num_filters]))
    filter1 = tf.get_variable("filter"+name,shape=conv_filt_shape,dtype = tf.float32,initializer = tf.contrib.layers.xavier_initializer_conv2d())
    conv1 = tf.nn.conv2d(X,filter1,strides=[1,1,1,1],padding = "SAME")
    conv1 += b1
    conv1 = tf.nn.relu(conv1)
    ksize = [1, pool_shape[0], pool_shape[1], 1]
    strides = [1, 2, 2, 1]
    pool1 = tf.nn.max_pool(conv1,ksize=ksize,strides=strides,padding="SAME")
    return pool1


In [26]:
tf.reset_default_graph()
learning_rate = tf.placeholder(tf.float32,[])
X = tf.placeholder(tf.float32,[None,200,200,3])
Y = tf.placeholder(tf.float32,[None,total_classes])
conv1 = convLayer(X,3,64,[3,3],[2,2],"1")
print (conv1.shape)
conv2 = convLayer(conv1,64,64,[3,3],[2,2],"2")
print (conv2.shape)
conv3 = convLayer(conv2,64,32,[3,3],[2,2],"3")
print (conv3.shape)
conv4 = convLayer(conv3,32,16,[3,3],[2,2],"4")
print (conv4.shape)
conv5 = convLayer(conv4,16,8,[3,3],[2,2],"5")
print (conv5.shape)
flattened = tf.reshape(conv5,[-1,7*7*8])

dense1 = tf.layers.dense(flattened,total_classes)

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = dense1,labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(loss)

a = tf.argmax(dense1,axis=1)
b = tf.argmax(Y,axis=1)
prediction = tf.equal(a,b)
accuracy = tf.reduce_mean(tf.cast(prediction,tf.float32))

saver = tf.train.Saver()

(?, 100, 100, 64)
(?, 50, 50, 64)
(?, 25, 25, 32)
(?, 13, 13, 16)
(?, 7, 7, 8)


In [ ]:
epochs = 180
init = tf.global_variables_initializer()
lr = 0.001
#valx=images[:30]
#valy=encoded_labels[:30]
#trainx=images[30:]
#trainy=encoded_labels[30:]
trainx,valx,trainy,valy=train_test_split(images,encoded_labels,test_size=30,stratify=encoded_labels)
best_acc = 0.0
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
sess.run(init)

for i in range(epochs):
    _,trainbatchx,_,trainbatchy=train_test_split(trainx,trainy,test_size=20, stratify=trainy)
    sess.run(optimizer,feed_dict={X:trainbatchx,Y:trainbatchy,learning_rate:lr})
    if (i%5==0):
        loss_value,acc = sess.run([loss,accuracy],feed_dict={X:trainbatchx,Y:trainbatchy})
        val_acc = sess.run(accuracy,feed_dict={X:valx,Y:valy})        
        if (val_acc > best_acc):
            saver.save(sess,"./Models/best_model10.0.ckpt")
            best_acc = val_acc
        print ("Loss: ",loss_value," Training Accuracy: ",int(acc*100),"%", " Validation Accuracy: ",int(val_acc*100),"%")    


Loss:  9.33334  Training Accuracy:  25 %  Validation Accuracy:  13 %
Loss:  3.14815  Training Accuracy:  34 %  Validation Accuracy:  33 %
Loss:  1.83768  Training Accuracy:  40 %  Validation Accuracy:  36 %
Loss:  1.69471  Training Accuracy:  40 %  Validation Accuracy:  36 %
Loss:  1.12472  Training Accuracy:  34 %  Validation Accuracy:  56 %
Loss:  0.594929  Training Accuracy:  80 %  Validation Accuracy:  53 %
Loss:  0.696625  Training Accuracy:  75 %  Validation Accuracy:  69 %
Loss:  0.701873  Training Accuracy:  80 %  Validation Accuracy:  69 %
